In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

os.environ['TORCH_USE_CUDA_DSA'] = "TRUE"
os.environ['CUDA_LAUNCH_BLOCKING']="TRUE"

In [ ]:
from use_case.baseline import * 
from tests.eval import *

np.random.seed(32042)
torch.manual_seed(1337)

N = 10
payoff_i = np.random.uniform(-10, 10, (N, N))
payoff_j = np.random.uniform(-10, 10, (N, N))

# Initialize environment
N_ACTIONS = payoff_i.shape[0]
N_AGENTS = 500
env = BaselineEnvironment(N_AGENTS, payoff_i, payoff_j, total_games = 5)

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import matplotlib.pyplot as plt

# Calculate average rewards for all action pairs
average_rewards = (payoff_i + payoff_j) / 2

# Flatten the matrix into a list of all possible rewards
all_rewards = average_rewards.ravel()

# Plotting the histogram
plt.figure(figsize=(10, 6))
plt.hist(all_rewards, bins=50, color='skyblue', edgecolor='black', alpha=0.7)
plt.title('Distribution of Average Rewards for All Action Pairs')
plt.xlabel('Average Reward')
plt.ylabel('Frequency')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:

# Calculate average rewards for all action pairs
max_rewards = np.max([payoff_i, payoff_j], axis = 0)

# Flatten the matrix into a list of all possible rewards
all_rewards = max_rewards.ravel()

# Plotting the histogram
plt.figure(figsize=(10, 6))
plt.hist(all_rewards, bins=50, color='skyblue', edgecolor='black', alpha=0.7)
plt.title('Distribution of Average Rewards for All Action Pairs')
plt.xlabel('Average Reward')
plt.ylabel('Frequency')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Actual Run

In [ ]:
from models.model import *
from models.trainer import *

In [ ]:
# Configure the network here
parameters = ParameterSettings(
    n_agents = N_AGENTS,
    d_action = N_ACTIONS, 
    d_obs = env.obs_size, 
    d_traits = env.d_traits,
    d_het_latent = 4, 
    d_beliefs = env.d_beliefs,
    d_relation = env.d_relation
)
parameters.device = "cuda" if torch.cuda.is_available() else "cpu"
model = PPOModel(parameters)

In [ ]:
equilibriua = find_pure_equilibria(payoff_i, payoff_j)

for eq in equilibriua:
    x, y = eq 
    a = (y[0] + y[1]) / 2

    print(x, a)

In [ ]:
evaluate_policy(model, env, 10)

In [ ]:
# Setup the training loop
training_parameters = TrainingParameters(
    outer_loops = 2_000,
    
    actor_learning_rate= 1e-4,
    critic_learning_rate = 1e-3,
    hypernet_learning_rate = 5e-4,

    hypernet_jsd_threshold = 2.0,
    hypernet_samples = 3000,
    hypernet_jsd_weight = 1.0,
    hypernet_entropy_weight = 1.0, 
    hypernet_diversity_weight= 1.0,

    sampled_agents_proportion = 0.1,
    experience_sampling_steps = 10,
    experience_buffer_size = 100,

    entropy_coeff = 0.2,

    epsilon_period = 200,
    epsilon_end = 0.2,

    entropy_target = 0.5,

    eval_temp = -1.0,
    
    verbose = False,
    device = parameters.device
)
train_model(model, env, training_parameters)
        

# Heterogeneous Baseline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from use_case.baseline import * 
from tests.eval import *
from models.model import *
from models.trainer import *


In [4]:
np.random.seed(1337)
torch.manual_seed(1337)

N_AGENTS = 1000
N_TYPES = 5
N_ACTIONS = 10
type_payoffs = np.random.uniform(-10, 10, (N_TYPES, N_TYPES, 2, N_ACTIONS, N_ACTIONS))
env = BaselineHeterogeneous(N_AGENTS, N_TYPES, type_payoffs, total_games = 4)

In [ ]:
# Configure the network here
parameters = ParameterSettings(
    n_agents = N_AGENTS,
    d_action = N_ACTIONS, 
    d_obs = env.obs_size, 
    d_traits = env.d_traits,
    d_het_latent = 4, 
    d_beliefs = env.d_beliefs,
    d_relation = env.d_relation,
    d_comm_state= env.d_comm_state,
    device = "cpu"
)

model = PPOModel(parameters)

In [23]:
# Setup the training loop
training_parameters = TrainingParameters(
    outer_loops = 10_000,
    
    actor_learning_rate= 1e-4,
    critic_learning_rate = 1e-3,
    hypernet_learning_rate = 5e-4,

    hypernet_jsd_threshold = 0.5,
    hypernet_samples = 10000,
    hypernet_jsd_weight = 1.0,
    hypernet_entropy_weight = 1.0, 
    hypernet_diversity_weight= 1.0,

    sampled_agents_proportion = 0.5,
    experience_sampling_steps = 10,
    experience_buffer_size = 100,

    entropy_coeff = 0.2,

    epsilon_period = 1000,
    epsilon_end = 0.2,

    entropy_target = 0.5,

    eval_temp = -1.0,
    
    verbose = False,
    device = parameters.device,
    eval_k= N_TYPES,
    checkpoint_interval = 100
)
train_model(model, env, training_parameters)


  0%|          | 1/10000 [00:06<16:58:56,  6.11s/it]


KeyboardInterrupt: 